In [1]:
from util import preprocessing_simple_train
import tensorflow as tf
from util import make_mean, month_to_day, make_dayhour_to_index

df = preprocessing_simple_train()

mean_df = make_mean(df)
mean_df['day'] = month_to_day(mean_df['rainfall_train.ef_month']) + mean_df['rainfall_train.ef_day']
mean_df = make_dayhour_to_index(mean_df)


2024-06-23 12:37:49.114870: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 12:37:49.640389: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 12:37:51.393669: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 12:37:56.391452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Simple LSTM Network


LSTM * 10

In [5]:
from util import make_day2class
import pandas as pd
from sklearn.preprocessing import StandardScaler

from util import make_day2class, make_dayhour_to_index
import numpy as np


v_year = "C"

train_df = mean_df[mean_df['rainfall_train.ef_year'] != v_year].copy()
test_df = mean_df[mean_df['rainfall_train.ef_year'] == v_year].copy()

day2class = make_day2class(train_df)
day2class = day2class.rename(columns={'rainfall_train.class_interval':'mean_class'})
day2class['mean_class'] = StandardScaler().fit_transform(day2class[['mean_class']])

train_df = pd.merge(train_df, day2class, how='left',on=['day'])
test_df = pd.merge(test_df, day2class, how='left',on=['day'])



#shape (_num,13)

_num = 30

feature_list = ['rainfall_train.ef_hour', 'rainfall_train.v01', 'rainfall_train.v02',
       'rainfall_train.v03', 'rainfall_train.v04', 'rainfall_train.v05',
       'rainfall_train.v06', 'rainfall_train.v07', 'rainfall_train.v08',
       'rainfall_train.v09', 'mean_vmedian', 'mean_class']

X = []
Y = []
for i in train_df['rainfall_train.ef_year'].unique():
    for j in range(1,21):
        tmp = train_df[(train_df['rainfall_train.ef_year'] == i) & (train_df['rainfall_train.stn4contest'] == f"STN{format(j,'03')}")].sort_values(['index']).copy()
        tmp['rainfall_train.ef_hour'] = tmp['rainfall_train.ef_hour'] / 24.0
        tmp_x = tmp[feature_list].values
        Y.extend(tmp['rainfall_train.class_interval'].values.astype(int))
        for k in range(1,_num):
            init = tmp_x[0:k]
            X.append(np.vstack([init,np.repeat(init[-1,:].reshape(1,len(feature_list)),repeats=_num-k, axis=0)]))
        m = len(tmp_x) - (_num-1)
        for k in range(m):
            X.append(tmp_x[k:k+_num].reshape(_num,len(feature_list)))
X = np.array(X)
Y = np.array(Y)
#-----------------#
X_V = []
Y_V = []
for i in test_df['rainfall_train.ef_year'].unique():
    for j in range(1,21):
        tmp = test_df[(test_df['rainfall_train.ef_year'] == i) & (test_df['rainfall_train.stn4contest'] == f"STN{format(j,'03')}")].sort_values(['index']).copy()
        tmp_x = tmp[feature_list].values
        Y_V.extend(tmp['rainfall_train.class_interval'].values)
        for k in range(1,_num):
            init = tmp_x[0:k]
            X_V.append(np.vstack([init,np.repeat(init[-1,:].reshape(1,len(feature_list)),repeats=_num-k, axis=0)]))
        m = len(tmp_x) - (_num-1)
        for k in range(m):
            X_V.append(tmp_x[k:k+_num].reshape(_num,len(feature_list)))
X_V = np.array(X_V)
Y_V = np.array(Y_V)
X = X.astype(np.float32)
X_V = X_V.astype(np.float32)

Y_V = Y_V.astype(np.float32)
Y_V = Y_V.astype(np.float32)




In [6]:
from tensorflow import keras

input_X = keras.layers.Input((_num,len(feature_list)))
MX = keras.layers.LSTM(len(feature_list),return_sequences=True)(input_X)
for i in range(10):
    MX = keras.layers.LSTM(100,return_sequences=True)(MX)
    MX = keras.layers.Dropout(0.2)(MX)
MX = keras.layers.LSTM(len(feature_list))(MX)
out = keras.layers.Dense(10, activation='softmax')(MX)

model = keras.Model(input_X,out)
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [7]:

checkpoint_cb = keras.callbacks.ModelCheckpoint('simple_lstm_best-model.keras')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, 
                                                  restore_best_weights=True)
history = model.fit(X[Y!=0], Y[Y!=0], batch_size=20, epochs=15, validation_data=[X_V[Y_V!=0],Y_V[Y_V!=0]], verbose=1,callbacks=[checkpoint_cb])


Epoch 1/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 330s 648ms/step - accuracy: 0.1692 - loss: 2.1203 - val_accuracy: 0.1805 - val_loss: 2.0926
Epoch 2/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 297s 688ms/step - accuracy: 0.1759 - loss: 2.0855 - val_accuracy: 0.1805 - val_loss: 2.0908
Epoch 3/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 297s 687ms/step - accuracy: 0.1820 - loss: 2.0858 - val_accuracy: 0.1805 - val_loss: 2.0902
Epoch 4/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 308s 655ms/step - accuracy: 0.1855 - loss: 2.0739 - val_accuracy: 0.1805 - val_loss: 2.0930
Epoch 5/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 318s 647ms/step - accuracy: 0.1871 - loss: 2.0865 - val_accuracy: 0.1805 - val_loss: 2.0927
Epoch 6/15
432/432 ━━━━━━━━━━━━━━━━━━━━ 301s 696ms/step - accuracy: 0.1827 - loss: 2.0779 - val_accuracy: 0.1805 - val_loss: 2.0902
Epoch 7/15
 85/432 ━━━━━━━━━━━━━━━━━━━━ 3:31 609ms/step - accuracy: 0.1999 - loss: 2.0741

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['csi'])
plt.plot(history.history['val_csi'])
plt.xlabel('epoch')
plt.ylabel('value')
plt.legend(['csi', 'val_csi'])
plt.savefig('7_3-07', dpi=300)
plt.show()